In [1]:
"""
Data Parser for recorded IB data

Author Derek Wong

Date: 2016-04-05
"""

import pandas as pd
import time

min_src_path = "C:\\Users\\treyd_000\\Desktop\\Quantinsti\\Azure-IB\\Azure-IB\\ohlc.csv"
tick_src_path = "C:\\Users\\treyd_000\\Desktop\\Quantinsti\\Azure-IB\\Azure-IB\\data.csv"

class DataParser:

    def __init__(self):
        pass

    def min_data_to_panda(self, src_path):
        start_time = time.clock()
        raw = pd.read_csv(src_path, sep=",",
                          names=["Datetime", "Open", "High", "Low", "Close", "Volume", "Count"],
                          skiprows=1)
        index = pd.to_datetime(raw.Datetime)
        raw.index = index
        clean = raw.drop(["Datetime"], 1)
        end_time = time.clock()
        print("Import Complete. ", clean.shape[0], " Lines Read in {:.2f}".format(end_time - start_time), "sec")
        return clean

    def tick_data_to_panda(self, src_path, padding = True):
        start_time = time.clock()
        raw = pd.read_csv(src_path, sep=",",
                          names=["Datetime", "Price", "Size", "Ask_price", "Ask_size", "Bid_price", "Bid_size"],
                          skiprows=1)
        index = pd.to_datetime(raw.Datetime)
        raw.index = index
        clean = raw.drop(["Datetime"], 1)
        if padding == True:
           clean.fillna(method="pad", inplace=True)
           clean.dropna(axis=0, inplace=True)
        end_time = time.clock()
        print("Import Complete. ", clean.shape[0], " Lines Read in {:.2f}".format(end_time - start_time), "sec")
        return clean

In [2]:
parser = DataParser()
min_data = parser.min_data_to_panda(min_src_path)

('Import Complete. ', 43, ' Lines Read in 0.00', 'sec')


In [3]:
tick_data = parser.tick_data_to_panda(tick_src_path)

('Import Complete. ', 182240, ' Lines Read in 0.18', 'sec')


In [4]:
"""
Z score trading algorithm logic

updated: 2016-04-04

Author: Derek Wong
"""
#update v0.1 added trend logic. simulated flags in dataframe. accepted flags for algo logic switching
#update v0.2 refactored into a class and takes bbo. use decimal package to prevent float errors

import decimal

FIVEPLACES = decimal.Decimal("0.00001")

class Zscore:
    """
    Zscore class function
    initialize takes:
        init_bid = initial bid value
        init_ask = initial ask value
        init_zscore = initial zscore
        init_mean = initial mean value
        init_state = initial state ("FLAT", "LONG", "SHORT")
        init_flag = initial flag ("trend", "range")
    """
    def __init__(self, init_bid, init_ask, init_zscore, init_mean, init_stdev, init_state, init_flag):
        self.bid = decimal.Decimal(init_bid).quantize(FIVEPLACES)
        self.ask = decimal.Decimal(init_ask).quantize(FIVEPLACES)
        self.zscore = decimal.Decimal(init_zscore).quantize(FIVEPLACES)
        self.mean = decimal.Decimal(init_mean).quantize(FIVEPLACES)
        self.stdev = decimal.Decimal(init_stdev).quantize(FIVEPLACES)
        self.state = init_state
        self.flag = init_flag
        self.mid_price = self.calc_mid_price(init_bid, init_ask)
        # initialize signals to NONE
        self.signal = "NONE"

        # initialize historical values same as current values at initialization
        self.hist_bid = self.bid
        self.hist_ask = self.ask
        self.hist_zscore = self.zscore
        self.hist_mean = self.mean
        self.hist_flag = self.flag
        self.hist_stdev = self.stdev

        #initialize internal variables
        self.state = "FLAT"
        self.signal = "NONE"

        self.hist_state = self.state
        self.hist_signal = self.signal

    def set_parameters(self, n_sma=30, n_stdev=30, z_threshold=2, z_close_thresh=0.2):
        # parameter initializations if we decide to calculate internally
        self.n_sma = n_sma
        self.n_stdev = n_stdev
        self.z_threshold = z_threshold
        self.z_close_thresh = z_close_thresh

    def calc_mid_price(self, bid, ask):
        # update the mid price by using average of bid/ask
        mid_price = decimal.Decimal((bid + ask) / 2).quantize(FIVEPLACES)
        self.mid_price = mid_price
        return mid_price

    def update_bbo(self, new_bid, new_ask):
        # update the bbo and mid price given the new values. store previous values
        self.hist_bid = self.bid
        self.hist_ask = self.ask

        self.bid = decimal.Decimal(new_bid).quantize(FIVEPLACES)
        self.ask = decimal.Decimal(new_ask).quantize(FIVEPLACES)

        self.mid_price = self.calc_mid_price(bid=new_bid, ask=new_ask)

    def update_mean_stdev(self, new_mean, new_stdev):
        # update the mean & stdev possibly from the 1 minute data. store previous values

        self.hist_mean = self.mean
        self.mean = new_mean

        self.hist_stdev = self.stdev
        self.stdev = new_stdev

    def update_flag(self, new_flag):
        # update the flag
        self.hist_flag = self.flag
        self.flag = new_flag

    def calc_zscore(self, mid_price):
        new_zscore = mid_price - decimal.Decimal(self.mean) / decimal.Decimal(self.stdev)
        self.zscore = new_zscore

    def algo_calc(self):
        # Enter Long Position Signal in trend flag
        if self.zscore > self.z_threshold and \
                self.hist_zscore <= self.z_threshold and \
                self.flag == "trend" and \
                self.state == "FLAT":

            self.hist_state = self.state
            self.hist_signal = self.signal

            self.signal = "BOT"
            self.state = "LONG"

        # Enter Short Position Signal in trend flag
        elif self.zscore < -self.z_threshold and \
                self.zscore >= -self.z_threshold and \
                self.flag == "trend" and \
                self.state == "FLAT":

            self.hist_state = self.state
            self.hist_signal = self.signal

            self.signal = "SLD"
            self.state = "SHORT"

        # Close Long position in trend flag
        elif self.hist_state == "LONG" and \
                self.zscore <= self.z_close_thresh and \
                self.hist_state == "trend":

            self.hist_state = self.state
            self.hist_signal = self.signal

            self.signal = "SLD"
            self.state = "FLAT"

        # Close Short Position in trend flag
        elif self.hist_state == "SHORT" and \
                self.zscore >= -self.z_close_thresh and\
                self.flag == "trend":

            self.hist_state = self.state
            self.hist_signal = self.signal

            self.signal = "BOT"
            self.state = "FLAT"

        # Enter Long Position Signal in range flag
        elif self.zscore < -self.z_threshold and \
                self.hist_zscore >= -self.z_threshold and \
                self.flag == "range" and \
                self.hist_state == "FLAT":

                self.hist_state = self.state
                self.hist_signal = self.signal

                self.signal = "BOT"
                self.state = "LONG"

        # Enter Short Position Signal in range flag
        elif self.zscore > self.z_threshold and \
                self.hist_zscore <= self.z_threshold and \
                self.flag == "range" and \
                self.hist_state == "FLAT":

                self.hist_state = self.state
                self.hist_signal = self.signal

                self.signal = "SLD"
                self.state = "SHORT"

        # Close Long position in range flag
        elif self.hist_state == "LONG" and \
                self.zscore >= -self.z_close_thresh and \
                self.flag == "range":

            self.hist_state = self.state
            self.hist_signal = self.signal

            self.signal = "SLD"
            self.state = "FLAT"

        # Close Short Position in range flag
        elif self.hist_state == "SHORT" and \
                        self.zscore <= self.z_close_thresh and \
                        self.flag == "range":

            self.hist_state = self.state
            self.hist_signal = self.signal

            self.signal = "BOT"
            self.state = "FLAT"

    def on_tick(self, cur_bid, cur_ask):
        # every tick pass the bid ask, perform calcs

        # take current bid ask and calculate mid price
        self.calc_mid_price(bid=decimal.Decimal(cur_bid), ask=decimal.Decimal(cur_ask))

        # calculate the new z score for the given tick mid price
        self.calc_zscore(self.mid_price)

        # run algo calc based on object self values
        self.algo_calc()

    def on_minute(self, new_mean, new_stdev, new_flag):
        # update the parameters every minute

        self.update_mean_stdev(new_mean=new_mean, new_stdev=new_stdev)
        self.update_flag(new_flag=new_flag)







In [5]:
min_data["sma"] = min_data.Close.rolling(window=30).mean()
min_data.tail()

,Open,High,Low,Close,Volume,Count,sma
Datetime,,,,,,,
2016-04-04 06:38:00,37.79,37.79,37.79,37.79,2.0,2.0,37.764000
2016-04-04 06:39:00,37.79,37.81,37.79,37.81,10.0,4.0,37.764000
2016-04-04 06:40:00,37.81,37.81,37.81,37.81,0.0,0.0,37.766000
2016-04-04 06:41:00,37.79,37.79,37.79,37.79,1.0,1.0,37.767333
2016-04-04 06:42:00,37.81,37.82,37.81,37.82,9.0,2.0,37.769000


In [6]:
min_data["stdev"] = min_data.Close.rolling(window=30).std()
min_data.tail()

,Open,High,Low,Close,Volume,Count,sma,stdev
Datetime,,,,,,,,
2016-04-04 06:38:00,37.79,37.79,37.79,37.79,2.0,2.0,37.764000,0.020103
2016-04-04 06:39:00,37.79,37.81,37.79,37.81,10.0,4.0,37.764000,0.020103
2016-04-04 06:40:00,37.81,37.81,37.81,37.81,0.0,0.0,37.766000,0.021592
2016-04-04 06:41:00,37.79,37.79,37.79,37.79,1.0,1.0,37.767333,0.021804
2016-04-04 06:42:00,37.81,37.82,37.81,37.82,9.0,2.0,37.769000,0.023831


In [7]:
min_data["zscore"] = (min_data.Close - min_data.sma) / min_data.stdev

In [8]:
min_data.dropna(axis=0, inplace=True)
min_data.tail()

,Open,High,Low,Close,Volume,Count,sma,stdev,zscore
Datetime,,,,,,,,,
2016-04-04 06:38:00,37.79,37.79,37.79,37.79,2.0,2.0,37.764000,0.020103,1.293328
2016-04-04 06:39:00,37.79,37.81,37.79,37.81,10.0,4.0,37.764000,0.020103,2.288195
2016-04-04 06:40:00,37.81,37.81,37.81,37.81,0.0,0.0,37.766000,0.021592,2.037808
2016-04-04 06:41:00,37.79,37.79,37.79,37.79,1.0,1.0,37.767333,0.021804,1.039578
2016-04-04 06:42:00,37.81,37.82,37.81,37.82,9.0,2.0,37.769000,0.023831,2.140042


In [9]:
myZ = Zscore(tick_data.Bid_price[1], tick_data.Ask_price[1], min_data.zscore[1], min_data.sma[1], min_data.stdev[1], "FLAT", "range")
myZ.set_parameters()
myZ

<__main__.Zscore instance at 0x00000000088E6F08>

In [10]:
myZ.on_tick(tick_data.Bid_price[2], tick_data.Ask_price[2])

In [11]:
myZ.on_minute(min_data.sma[2], min_data.stdev[2], "trend")

In [12]:
print myZ.mean, myZ.zscore, myZ.flag, myZ.bid, myZ.ask

37.7646666667 -697.1632684824902723735408560 trend 37.76000 37.77000


In [13]:
tick_data.head()

,Price,Size,Ask_price,Ask_size,Bid_price,Bid_size
Datetime,,,,,,
2016-04-05 00:29:37.413255,37.68,6.0,37.77,27.0,37.76,7.0
2016-04-05 00:29:37.415105,37.68,6.0,37.77,27.0,37.76,7.0
2016-04-05 00:29:37.417611,37.68,6.0,37.77,27.0,37.67,7.0
2016-04-05 00:29:37.418922,37.68,6.0,37.77,27.0,37.67,24.0
2016-04-05 00:29:37.422408,37.68,6.0,37.68,27.0,37.67,24.0
